In [3]:
#!/usr/bin/env python3
import os
from pathlib import Path
import pandas as pd
import numpy as np

"""
===============================================================================
FUTURES BACKTEST ENGINE WITH RISK-BASED POSITION SIZING
WITH MICRO/MINI TRADE MAPPING
===============================================================================

Three Position Sizing Approaches:
1) CLOSED EQUITY: Size off yesterday's ending equity (conservative)
   - Only realized P&L affects sizing
   - Unrealized gains don't inflate position sizes
   
2) OPEN EQUITY: Size off current equity including unrealized P&L
   - Most aggressive
   - Can pyramid into winning positions
   
3) RISK-ADJUSTED EQUITY: Size off closed equity minus open risks
   - Most conservative
   - Accounts for worst-case if all stops hit

Position Sizing Formula:
    Risk_$ = equity_base × risk_per_trade_pct
    Stop_distance = k × ewma32_std (k typically 2-4)
    Risk_per_contract = stop_distance × multiplier × FX
    Contracts = floor(Risk_$ / risk_per_contract)

Portfolio Heat Cap (optional):
    Sum of all open risks ≤ max_portfolio_heat × closed_equity

Signals/vol/price:
- Always computed from FULL-SIZE "signal symbol" data files.

Sizing + P&L:
- Use MICRO/MINI "trade symbol" multiplier (point_value) when available per mapping.
- Price changes are still taken from the signal symbol (assumes micro tracks underlying).

P&L:
    pnl_t = sum_i contracts_{i,t-1} * multiplier_trade_i * FX * (close_signal_t - close_signal_{t-1})

Outputs:
- portfolio_equity.(csv|parquet)
- positions.(csv|parquet)              # positions expressed in TRADE contracts (e.g., MES)
- trades.csv                            # contract changes in TRADE contracts
- instrument_map.csv                    # signal_symbol -> trade_symbol -> multiplier
===============================================================================
"""

# ============================================================
# Configuration
# ============================================================

# ==================== MICRO/MINI CONTRACT MAPPING ====================
# Maps full-size signal symbol -> micro/mini trade symbol
# Only includes mappings where the micro/mini EXISTS and is commonly traded

MICRO_MINI_MAPPING = {
    # Equity Indices - Micros available in Norgate
    'ES': 'MES',    # E-mini S&P 500 -> Micro E-mini S&P 500
    'NQ': 'MNQ',    # E-mini Nasdaq -> Micro E-mini Nasdaq
    'RTY': 'M2K',   # E-mini Russell 2000 -> Micro E-mini Russell
    'YM': 'MYM',    # E-mini Dow -> Micro E-mini Dow

    # Crypto - Micros available in Norgate
    'BTC': 'MBT',   # Bitcoin -> Micro Bitcoin
    'ETH': 'MET',   # Ether -> Micro Ether
}

# Additional mappings for contracts where micro EXISTS at broker but NOT in Norgate
# These use full-size data for signals but micro specs for sizing/P&L
BROKER_ONLY_MICROS = {
    # Currencies
    '6A': 'M6A',
    '6B': 'M6B',
    '6C': 'MCD',
    '6E': 'M6E',
    '6J': 'MJY',
    '6S': 'MSF',

    # Energy
    'CL': 'MCL',
    'NG': 'MNG',
    'RB': 'MRB',
    'HO': 'MHO',

    # Metals
    'GC': 'MGC',
    'SI': 'SIL',
    'HG': 'MHG',

    # Agriculture
    'ZC': 'MZC',
    'ZW': 'MZW',
    'ZS': 'MZS',
    'ZM': 'MZM',
    'ZL': 'MZL',

    # Volatility
    'VX': 'VXM',
}

# Inverse map: micro/mini trade symbol -> full-size signal symbol
# Used to ensure we do NOT accidentally use micro files (e.g., MES.parquet) as signal sources
MICRO_TO_SIGNAL = {v: k for k, v in MICRO_MINI_MAPPING.items()}

print(f"✓ Micro mappings defined")

CONFIG = {
    # Limit trading universe by TRADE symbol (e.g., {"MES"} to trade only Micro ES)
    #"trade_symbol_allowlist": {"MES"},  # set to None to disable filtering
    "trade_symbol_allowlist": None,  # set to None to disable filtering

    # USD Risk Volume Filtering
    "min_usd_risk_volume": 0,  # Minimum USD risk volume (0 = all instruments)

    # Backtest range (None means use all available)
    "start_date": "2015-01-01",
    "end_date": None,
 
    # Capital
    "initial_capital": 400_000.0,
    "trading_days_per_year": 256,
    "fx": 1.0,

    # ============================================================
    # RISK-BASED POSITION SIZING PARAMETERS (NEW)
    # ============================================================
    
    # Choose equity approach for position sizing:
    # "closed"          = Use yesterday's ending equity (conservative)
    # "open"            = Use current equity including unrealized P&L (aggressive)
    # "risk_adjusted"   = Use closed equity minus open risks (most conservative)
    "equity_method": "closed",  # Options: "closed", "open", "risk_adjusted"
    
    # Risk per trade as % of equity base
    "risk_per_trade_pct": 0.005,  # 0.5% of equity per trade (0.0025-0.01 typical range)
    
    # Stop loss distance (multiplier of daily volatility)
    "stop_loss_k": 3.0,  # Stop = entry ± k × ewma32_std (2-4 typical range)
    
    # Portfolio heat cap (optional)
    # Max total risk across all open positions as % of closed equity
    # Set to None to disable, or e.g. 0.10 for 10% max total risk
    "max_portfolio_heat": 0.10,  # 10% max total portfolio risk
    
    # Minimum contracts (set to 1 to require at least 1 contract per trade)
    "min_contracts": 1,
    
    # ============================================================

    # Signal params (trend strategy)
    "fast_sma": 50,
    "slow_sma": 100,

    # Costs (optional)
    "commission_per_contract": 2.50,  # per trade contract changed
    "apply_commissions": True,

    # Data locations
    "contracts_file": Path("./01-futures_universe/futures_contracts_full.parquet"),

    # IMPORTANT:
    # This directory contains the FULL-SIZE signal symbol data with ewma32_std, close, and usd_risk_volume
    "signal_vol_prices_dir": Path("./04a-risk_volume/norgate_continuous/parquet"),

    # Column names in per-instrument files
    "date_col": "date",
    "close_col": "close",
    "vol_col": "ewma32_std",  # DAILY EWMA stdev of returns

    # Output root
    "output_root": Path("./05a-futures_backtest_risk_based"),

    # Behavior
    "require_trade_multiplier": True,  # if True, error if mapped trade symbol has no multiplier
    "default_to_signal_multiplier_if_missing": True,  # if trade multiplier missing, use signal multiplier (if allowed)
}

print(f"✓ Configuration loaded")
print(f"  Equity method: {CONFIG['equity_method']}")
print(f"  Risk per trade: {CONFIG['risk_per_trade_pct']*100:.2f}%")
print(f"  Stop loss: {CONFIG['stop_loss_k']}× daily volatility")
if CONFIG['max_portfolio_heat']:
    print(f"  Max portfolio heat: {CONFIG['max_portfolio_heat']*100:.0f}%")

# ============================================================
# Helpers
# ============================================================

def load_contract_multipliers(contracts_path: Path) -> dict:
    df = pd.read_parquet(contracts_path)
    df.columns = [str(c).strip().lower() for c in df.columns]

    sym_col = next((c for c in ["symbol", "ticker", "root"] if c in df.columns), None)
    mult_col = next((c for c in ["point_value", "pointvalue", "multiplier", "contract_multiplier"] if c in df.columns), None)

    if sym_col is None or mult_col is None:
        raise ValueError(f"Missing symbol or multiplier columns. Found: {list(df.columns)}")

    multipliers = {}
    for _, row in df.iterrows():
        sym = str(row[sym_col]).strip().upper()
        mult = pd.to_numeric(row[mult_col], errors='coerce')
        if pd.notna(mult):
            multipliers[sym] = float(mult)
    return multipliers

def map_trade_symbol(signal_symbol: str) -> str:
    """Map signal symbol (e.g., ES) to trade symbol (e.g., MES if available)."""
    signal_symbol = signal_symbol.strip().upper()
    
    # Check Norgate micros first
    if signal_symbol in MICRO_MINI_MAPPING:
        return MICRO_MINI_MAPPING[signal_symbol]
    
    # Check broker-only micros
    if signal_symbol in BROKER_ONLY_MICROS:
        return BROKER_ONLY_MICROS[signal_symbol]
    
    # Default: trade the full-size symbol
    return signal_symbol

def load_instrument_df(path: Path, date_col: str, close_col: str, vol_col: str) -> pd.DataFrame:
    df = pd.read_parquet(path)
    df.columns = [str(c).strip().lower() for c in df.columns]

    if date_col not in df.columns:
        if isinstance(df.index, pd.DatetimeIndex):
            df = df.reset_index().rename(columns={"index": date_col})
        else:
            raise ValueError(f"Missing '{date_col}' in {path.name}. Columns: {list(df.columns)}")

    df[date_col] = pd.to_datetime(df[date_col], errors="coerce")
    df = df.dropna(subset=[date_col]).sort_values(date_col)

    df[close_col] = pd.to_numeric(df[close_col], errors="coerce")
    df[vol_col] = pd.to_numeric(df[vol_col], errors="coerce")
    
    # Convert usd_risk_volume if it exists
    if 'usd_risk_volume' in df.columns:
        df['usd_risk_volume'] = pd.to_numeric(df['usd_risk_volume'], errors='coerce')
        return df[[date_col, close_col, vol_col, 'usd_risk_volume']].copy()
    else:
        return df[[date_col, close_col, vol_col]].copy()

def compute_signals(close: pd.Series, fast: int, slow: int) -> pd.Series:
    sma_fast = close.rolling(window=fast, min_periods=fast).mean()
    sma_slow = close.rolling(window=slow, min_periods=slow).mean()

    sig = pd.Series(0, index=close.index, dtype=int)
    sig[sma_fast > sma_slow] = 1
    sig[sma_fast < sma_slow] = -1
    return sig

def safe_floor(x: float) -> int:
    if np.isnan(x) or np.isinf(x):
        return 0
    return int(np.floor(x))

# ============================================================
# Risk-Based Position Sizing Functions (NEW)
# ============================================================

def calculate_stop_distance(close_price: float, daily_vol: float, k: float) -> float:
    """
    Calculate stop loss distance in price points.
    
    Stop distance = k × daily_volatility
    
    Parameters:
    -----------
    close_price : Current close price
    daily_vol : Daily volatility (ewma32_std)
    k : Multiplier (e.g., 3.0 for 3× daily vol)
    
    Returns:
    --------
    Stop distance in price points
    """
    if pd.isna(close_price) or pd.isna(daily_vol) or daily_vol <= 0:
        return np.nan
    
    stop_distance = k * daily_vol * close_price  # In price points
    return float(stop_distance)

def calculate_risk_based_position_size(
    equity_base: float,
    risk_per_trade_pct: float,
    stop_distance: float,
    multiplier: float,
    fx: float,
    signal_direction: int,
    min_contracts: int = 1
) -> int:
    """
    Calculate position size based on risk per trade.
    
    Formula:
        Risk_$ = equity_base × risk_per_trade_pct
        Risk_per_contract = stop_distance × multiplier × FX
        Contracts = floor(Risk_$ / risk_per_contract)
    
    Parameters:
    -----------
    equity_base : Equity to base sizing on (closed/open/risk-adjusted)
    risk_per_trade_pct : % of equity to risk per trade (e.g., 0.005 for 0.5%)
    stop_distance : Stop loss distance in price points
    multiplier : Contract multiplier (point value)
    fx : FX conversion rate
    signal_direction : +1 for long, -1 for short, 0 for no position
    min_contracts : Minimum contracts per trade (default 1)
    
    Returns:
    --------
    Signed number of contracts (positive for long, negative for short)
    """
    if signal_direction == 0:
        return 0
    
    if equity_base <= 0 or pd.isna(stop_distance) or stop_distance <= 0:
        return 0
    
    # Dollar risk budget for this trade
    risk_dollars = equity_base * risk_per_trade_pct
    
    # Risk per contract
    risk_per_contract = stop_distance * multiplier * fx
    
    if risk_per_contract <= 0 or not np.isfinite(risk_per_contract):
        return 0
    
    # Calculate contracts
    contracts_float = risk_dollars / risk_per_contract
    contracts = safe_floor(contracts_float)
    
    # Apply minimum
    if contracts < min_contracts:
        return 0
    
    # Apply signal direction
    return int(signal_direction * contracts)

def calculate_open_risk(
    position_contracts: int,
    entry_price: float,
    current_price: float,
    stop_distance: float,
    multiplier: float,
    fx: float
) -> float:
    """
    Calculate the dollar risk of an open position if stop is hit.
    
    For long: risk = (entry_price - (entry_price - stop_distance)) × contracts × mult × FX
    For short: risk = ((entry_price + stop_distance) - entry_price) × contracts × mult × FX
    
    Simplified: risk = stop_distance × abs(contracts) × mult × FX
    
    Parameters:
    -----------
    position_contracts : Signed number of contracts held
    entry_price : Entry price for the position
    current_price : Current market price
    stop_distance : Stop loss distance in price points
    multiplier : Contract multiplier
    fx : FX conversion rate
    
    Returns:
    --------
    Dollar risk (always positive)
    """
    if position_contracts == 0 or pd.isna(stop_distance) or stop_distance <= 0:
        return 0.0
    
    # Risk is the same for long or short: stop_distance × contracts × multiplier
    risk = stop_distance * abs(position_contracts) * multiplier * fx
    return float(risk)

def check_portfolio_heat_cap(
    current_open_risks: dict,
    new_trade_risk: float,
    closed_equity: float,
    max_heat_pct: float
) -> bool:
    """
    Check if adding a new trade would violate portfolio heat cap.
    
    Parameters:
    -----------
    current_open_risks : Dict of {symbol: open_risk_dollars}
    new_trade_risk : Dollar risk of proposed new trade
    closed_equity : Closed equity base
    max_heat_pct : Maximum allowed heat as % of closed equity (e.g., 0.10 for 10%)
    
    Returns:
    --------
    True if trade is allowed, False if it would violate heat cap
    """
    if max_heat_pct is None:
        return True  # No cap enforced
    
    total_current_risk = sum(current_open_risks.values())
    total_new_risk = total_current_risk + new_trade_risk
    max_allowed_risk = closed_equity * max_heat_pct
    
    return total_new_risk <= max_allowed_risk

# ============================================================
# Main Backtest Engine
# ============================================================

def run_backtest(cfg: dict) -> tuple[pd.DataFrame, pd.DataFrame, pd.DataFrame, pd.DataFrame]:
    date_col = cfg["date_col"]
    close_col = cfg["close_col"]
    vol_col = cfg["vol_col"]

    multipliers = load_contract_multipliers(cfg["contracts_file"])

    signal_dir: Path = cfg["signal_vol_prices_dir"]
    if not signal_dir.exists():
        raise FileNotFoundError(f"Signal vol/price parquet dir not found: {signal_dir}")

    signal_files = sorted(signal_dir.glob("*.parquet"))
    if not signal_files:
        raise RuntimeError(f"No signal parquet files found in: {signal_dir}")

    # Load signals from full-size files; map to trade symbol for multiplier + P&L sizing
    instruments = []
    data_map = {}
    instrument_meta_rows = []

    print("\nLoading instruments...")
    for p in signal_files:
        signal_sym = p.stem.strip().upper()

        # SAFEGUARD: If this file is a micro/mini symbol, skip it
        if signal_sym in MICRO_TO_SIGNAL:
            continue
            
        trade_sym = map_trade_symbol(signal_sym)

        # OPTION 1: Filter by TRADE symbol allowlist
        allow = cfg.get("trade_symbol_allowlist")
        if allow is not None and trade_sym not in set(allow):
            continue

        # Need signal data
        df = load_instrument_df(p, date_col, close_col, vol_col)
        
        # OPTION 2: Filter by USD risk volume
        min_usd_risk = cfg.get("min_usd_risk_volume", 0)
        if min_usd_risk > 0:
            if "usd_risk_volume" not in df.columns:
                continue
            
            usd_risk_series = df["usd_risk_volume"].dropna()
            if len(usd_risk_series) == 0:
                continue
            
            recent_usd_risk = float(usd_risk_series.iloc[-1])
            
            if recent_usd_risk < min_usd_risk:
                continue
        
        df = df.set_index(date_col)
        df["signal"] = compute_signals(df[close_col], cfg["fast_sma"], cfg["slow_sma"])

        # Determine multiplier
        trade_mult = multipliers.get(trade_sym)
        signal_mult = multipliers.get(signal_sym)

        chosen_mult = trade_mult

        if chosen_mult is None:
            if cfg.get("default_to_signal_multiplier_if_missing", True) and signal_mult is not None:
                chosen_mult = signal_mult
            elif cfg.get("require_trade_multiplier", True):
                raise RuntimeError(
                    f"Missing multiplier for trade symbol '{trade_sym}' (mapped from '{signal_sym}'). "
                )

        if chosen_mult is None:
            continue

        data_map[signal_sym] = df
        instruments.append(signal_sym)

        instrument_meta_rows.append({
            "signal_symbol": signal_sym,
            "trade_symbol": trade_sym,
            "multiplier_used": float(chosen_mult),
            "multiplier_source": "trade_symbol" if trade_mult is not None else "signal_symbol",
        })

    if not instruments:
        raise RuntimeError(
            "No instruments loaded. Check that your signal files exist and multipliers are available."
        )

    instrument_map_df = pd.DataFrame(instrument_meta_rows).sort_values(["signal_symbol", "trade_symbol"])
    print(f"✓ Loaded {len(instruments)} instruments")
    print(instrument_map_df.to_string(index=False))
    
    # Build common calendar
    common_dates = None
    for sym in instruments:
        idx = data_map[sym].index
        common_dates = idx if common_dates is None else common_dates.intersection(idx)
    common_dates = common_dates.sort_values()

    if cfg["start_date"]:
        common_dates = common_dates[common_dates >= pd.Timestamp(cfg["start_date"])]
    if cfg["end_date"]:
        common_dates = common_dates[common_dates <= pd.Timestamp(cfg["end_date"])]

    if len(common_dates) < 3:
        raise RuntimeError("Not enough common dates after filtering to run the backtest.")

    print(f"✓ Backtest period: {common_dates[0].date()} to {common_dates[-1].date()} ({len(common_dates)} days)")

    # Matrices
    close = pd.DataFrame({sym: data_map[sym].reindex(common_dates)[close_col] for sym in instruments})
    vol_d = pd.DataFrame({sym: data_map[sym].reindex(common_dates)[vol_col] for sym in instruments})
    sig = pd.DataFrame({sym: data_map[sym].reindex(common_dates)["signal"] for sym in instruments}).astype(int)

    # Multiplier per signal symbol
    mult_used = {}
    trade_symbol_used = {}
    for _, r in instrument_map_df.iterrows():
        s = r["signal_symbol"]
        mult_used[s] = float(r["multiplier_used"])
        trade_symbol_used[s] = str(r["trade_symbol"])

    # State
    equity = float(cfg["initial_capital"])
    closed_equity = float(cfg["initial_capital"])  # Tracks realized P&L only
    fx = float(cfg["fx"])
    comm = float(cfg["commission_per_contract"])
    apply_comm = bool(cfg["apply_commissions"])

    # Position tracking
    prev_pos = {sym: 0 for sym in instruments}
    entry_prices = {sym: np.nan for sym in instruments}  # Track entry prices for risk calculation
    stop_distances = {sym: np.nan for sym in instruments}  # Track stop distances

    # Logs
    portfolio_rows = []
    positions_rows = []
    trades_rows = []

    print(f"\nRunning backtest with {cfg['equity_method']} equity method...")
    
    for t in range(1, len(common_dates)):
        dt_prev = common_dates[t - 1]
        dt = common_dates[t]

        # --- 1) Realize P&L from positions held over (dt_prev -> dt) ---
        pnl = 0.0
        for sym in instruments:
            c0 = close.at[dt_prev, sym]
            c1 = close.at[dt, sym]
            if pd.isna(c0) or pd.isna(c1):
                continue
            mult = mult_used[sym]
            pnl += prev_pos[sym] * mult * fx * float(c1 - c0)

        equity_before_rebal = equity
        equity += pnl

        # Update closed equity (only updates when positions change - see below)
        
        # --- 2) Calculate equity base for position sizing ---
        if cfg["equity_method"] == "closed":
            equity_base = closed_equity  # Yesterday's ending equity
        elif cfg["equity_method"] == "open":
            equity_base = equity  # Current equity including unrealized P&L
        elif cfg["equity_method"] == "risk_adjusted":
            # Calculate total open risk
            open_risks = {}
            for sym in instruments:
                if prev_pos[sym] != 0:
                    open_risk = calculate_open_risk(
                        prev_pos[sym],
                        entry_prices[sym],
                        close.at[dt, sym],
                        stop_distances[sym],
                        mult_used[sym],
                        fx
                    )
                    open_risks[sym] = open_risk
            
            total_open_risk = sum(open_risks.values())
            equity_base = max(0, closed_equity - total_open_risk)
        else:
            raise ValueError(f"Unknown equity_method: {cfg['equity_method']}")

        # --- 3) Compute target positions for dt -> dt+1 ---
        new_pos = {}
        proposed_entry_prices = {}
        proposed_stop_distances = {}
        open_risks_current = {}  # Track current open risks for heat cap

        # First, calculate open risks for existing positions
        for sym in instruments:
            if prev_pos[sym] != 0:
                open_risk = calculate_open_risk(
                    prev_pos[sym],
                    entry_prices[sym],
                    close.at[dt, sym],
                    stop_distances[sym],
                    mult_used[sym],
                    fx
                )
                open_risks_current[sym] = open_risk

        for sym in instruments:
            s = int(sig.at[dt, sym])
            c = close.at[dt, sym]
            vd = vol_d.at[dt, sym]
            mult = mult_used[sym]

            # Calculate stop distance for this instrument
            stop_dist = calculate_stop_distance(c, vd, cfg["stop_loss_k"])

            if pd.isna(c) or pd.isna(vd) or pd.isna(stop_dist) or s == 0:
                new_pos[sym] = 0
                proposed_entry_prices[sym] = np.nan
                proposed_stop_distances[sym] = np.nan
                continue

            # Calculate risk-based position size
            N = calculate_risk_based_position_size(
                equity_base=equity_base,
                risk_per_trade_pct=cfg["risk_per_trade_pct"],
                stop_distance=stop_dist,
                multiplier=mult,
                fx=fx,
                signal_direction=s,
                min_contracts=cfg.get("min_contracts", 1)
            )

            # Check portfolio heat cap (if enabled)
            if cfg.get("max_portfolio_heat") and N != 0:
                proposed_risk = calculate_open_risk(N, c, c, stop_dist, mult, fx)
                
                # Exclude current symbol's risk (we're replacing it)
                other_risks = {k: v for k, v in open_risks_current.items() if k != sym}
                
                if not check_portfolio_heat_cap(other_risks, proposed_risk, closed_equity, cfg["max_portfolio_heat"]):
                    N = 0  # Reject trade due to heat cap

            new_pos[sym] = N
            proposed_entry_prices[sym] = c if N != 0 else np.nan
            proposed_stop_distances[sym] = stop_dist if N != 0 else np.nan

        # --- 4) Commissions on changes ---
        commissions = 0.0
        if apply_comm:
            for sym in instruments:
                delta = new_pos[sym] - prev_pos[sym]
                if delta != 0:
                    commissions += abs(delta) * comm

        equity -= commissions

        # --- 5) Update closed equity (realized P&L) ---
        # When positions change, we "realize" the P&L
        realized_pnl = 0.0
        for sym in instruments:
            if prev_pos[sym] != 0 and new_pos[sym] != prev_pos[sym]:
                # Position changed - realize the P&L
                realized_pnl += prev_pos[sym] * mult_used[sym] * fx * float(close.at[dt, sym] - entry_prices[sym])
        
        closed_equity += realized_pnl - commissions

        # --- 6) Log trades ---
        for sym in instruments:
            delta = new_pos[sym] - prev_pos[sym]
            if delta != 0:
                trades_rows.append({
                    "date": dt.date().isoformat(),
                    "signal_symbol": sym,
                    "trade_symbol": trade_symbol_used[sym],
                    "prev_contracts": int(prev_pos[sym]),
                    "new_contracts": int(new_pos[sym]),
                    "delta_contracts": int(delta),
                    "commission": float(abs(delta) * comm) if apply_comm else 0.0,
                    "signal": int(sig.at[dt, sym]),
                    "close_signal": float(close.at[dt, sym]) if pd.notna(close.at[dt, sym]) else np.nan,
                    "daily_vol": float(vol_d.at[dt, sym]) if pd.notna(vol_d.at[dt, sym]) else np.nan,
                    "stop_distance": float(proposed_stop_distances[sym]) if pd.notna(proposed_stop_distances[sym]) else np.nan,
                    "multiplier_trade": float(mult_used[sym]),
                    "equity_base": float(equity_base),
                    "risk_per_trade_pct": float(cfg["risk_per_trade_pct"]),
                })

        # Update entry prices and stop distances
        entry_prices = proposed_entry_prices.copy()
        stop_distances = proposed_stop_distances.copy()

        # Positions table
        pos_row = {"date": dt.date().isoformat()}
        for sym in instruments:
            col = trade_symbol_used[sym]
            pos_row[col] = int(new_pos[sym])
        positions_rows.append(pos_row)

        # Portfolio row
        ret = (equity - equity_before_rebal) / equity_before_rebal if equity_before_rebal != 0 else np.nan
        
        # Calculate total open risk
        total_open_risk = sum([
            calculate_open_risk(new_pos[sym], entry_prices[sym], close.at[dt, sym], 
                              stop_distances[sym], mult_used[sym], fx)
            for sym in instruments if new_pos[sym] != 0
        ])
        
        portfolio_rows.append({
            "date": dt.date().isoformat(),
            "equity": float(equity),
            "closed_equity": float(closed_equity),
            "unrealized_pnl": float(equity - closed_equity),
            "pnl": float(pnl),
            "commissions": float(commissions),
            "return": float(ret),
            "equity_base_used": float(equity_base),
            "total_open_risk": float(total_open_risk),
            "heat_pct": float(total_open_risk / closed_equity) if closed_equity > 0 else 0.0,
        })

        prev_pos = new_pos.copy()

    portfolio_df = pd.DataFrame(portfolio_rows)
    positions_df = pd.DataFrame(positions_rows).fillna(0)

    # Ensure deterministic column order for positions
    if not positions_df.empty:
        cols = ["date"] + sorted([c for c in positions_df.columns if c != "date"])
        positions_df = positions_df[cols]

    trades_df = pd.DataFrame(trades_rows)

    return portfolio_df, positions_df, trades_df, instrument_map_df

def write_outputs(cfg: dict, portfolio: pd.DataFrame, positions: pd.DataFrame, 
                 trades: pd.DataFrame, instrument_map: pd.DataFrame) -> None:
    out_root = Path(cfg["output_root"])
    out_root.mkdir(parents=True, exist_ok=True)

    # Add equity method to output path
    equity_method = cfg["equity_method"]
    out_dir = out_root / equity_method
    out_dir.mkdir(parents=True, exist_ok=True)

    portfolio.to_csv(out_dir / "portfolio_equity.csv", index=False)
    portfolio.to_parquet(out_dir / "portfolio_equity.parquet", index=False)

    positions.to_csv(out_dir / "positions.csv", index=False)
    positions.to_parquet(out_dir / "positions.parquet", index=False)

    trades.to_csv(out_dir / "trades.csv", index=False)

    instrument_map.to_csv(out_dir / "instrument_map.csv", index=False)

    print(f"\n✓ Outputs written to: {out_dir}")
    print(f"  - portfolio_equity.(csv|parquet)")
    print(f"  - positions.(csv|parquet)")
    print(f"  - trades.csv")
    print(f"  - instrument_map.csv")

def main():
    portfolio_df, positions_df, trades_df, instrument_map_df = run_backtest(CONFIG)
    write_outputs(CONFIG, portfolio_df, positions_df, trades_df, instrument_map_df)
    
    # Print summary statistics
    print(f"\n{'='*80}")
    print("BACKTEST SUMMARY")
    print(f"{'='*80}")
    print(f"Equity Method: {CONFIG['equity_method']}")
    print(f"Risk per Trade: {CONFIG['risk_per_trade_pct']*100:.2f}%")
    if CONFIG['max_portfolio_heat']:
        print(f"Max Portfolio Heat: {CONFIG['max_portfolio_heat']*100:.0f}%")
    print(f"\nStarting Equity: ${CONFIG['initial_capital']:,.0f}")
    print(f"Ending Equity: ${portfolio_df['equity'].iloc[-1]:,.0f}")
    total_return = (portfolio_df['equity'].iloc[-1] / CONFIG['initial_capital'] - 1) * 100
    print(f"Total Return: {total_return:+.2f}%")
    
    # Calculate volatility
    returns = portfolio_df['return'].dropna()
    if len(returns) > 1:
        daily_vol = returns.std()
        ann_vol = daily_vol * np.sqrt(CONFIG['trading_days_per_year'])
        print(f"Annualized Volatility: {ann_vol*100:.2f}%")
        
        mean_daily = returns.mean()
        sharpe = (mean_daily / daily_vol) * np.sqrt(CONFIG['trading_days_per_year']) if daily_vol > 0 else 0
        print(f"Sharpe Ratio: {sharpe:.2f}")
    
    print(f"\nTotal Trades: {len(trades_df)}")
    print(f"Total Commissions: ${portfolio_df['commissions'].sum():,.2f}")
    
    # Average heat
    avg_heat = portfolio_df['heat_pct'].mean() * 100
    max_heat = portfolio_df['heat_pct'].max() * 100
    print(f"\nAverage Portfolio Heat: {avg_heat:.2f}%")
    print(f"Maximum Portfolio Heat: {max_heat:.2f}%")
    print(f"{'='*80}")

if __name__ == "__main__":
    main()


✓ Micro mappings defined
✓ Configuration loaded
  Equity method: closed
  Risk per trade: 0.50%
  Stop loss: 3.0× daily volatility
  Max portfolio heat: 10%

Loading instruments...
✓ Loaded 99 instruments
signal_symbol trade_symbol  multiplier_used multiplier_source
           6A          M6A         100000.0     signal_symbol
           6B          M6B          62500.0     signal_symbol
           6C          MCD         100000.0     signal_symbol
           6E          M6E         125000.0     signal_symbol
           6J          MJY         125000.0     signal_symbol
           6M           6M         500000.0      trade_symbol
           6N           6N         100000.0      trade_symbol
           6S          MSF         125000.0     signal_symbol
          AFB          AFB             20.0      trade_symbol
          AWM          AWM             20.0      trade_symbol
          BRN          BRN           1000.0      trade_symbol
          BTC          MBT              0.1      tr